# Heading

This notebook is for testing orientation-detecting models.

In [6]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import sys

def letter_seg(path):
    #convert image to a numpy array
    img = Image.open(path)
    img = img.resize((224,224))
    img = np.array(img).astype(np.float32)
    Z = img.reshape(224*224,3)

    #use k-means to convert image to 2 colors
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = 2
    ret,label,center=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
    center = np.uint8(center)
    res = center[label.flatten()]
    res2 = res.reshape((img.shape))

    #convert the image to black and white
    res2 = (res2 == res2[0,0]).astype(np.float32)

    #create mask for floodfilling
    h, w = res2.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)

    #meant to close gaps but doesn't work well
    #kernel = np.ones((14,14),np.uint8)
    #res2 = cv2.morphologyEx(res2, cv2.MORPH_CLOSE, kernel)

    #Floodfill from point (0, 0)
    cv2.floodFill(res2, mask, (0,0), 0)
    
    return np.expand_dims(res2[:,:,0], axis=2)


In [7]:
import sys
from PIL import Image
import numpy as np
import cv2
import keras.models
sys.path.insert(0, 'segmentation')
import segmentation.letter_seg as ls

SHAPE_INDEX = [
    'circle', 'cross', 'heptagon', 'octagon', 'pentagon', 'quarter_circle',
    'rectangle', 'semicircle', 'star', 'trapezoid', 'triangle'
]


def shape_img(path):
    img = Image.open(path)
    img = img.resize((224, 224))
    #img = img.reshape((224, 224))
    img = np.array(img).astype(np.float32)
    img = np.expand_dims(img, axis=0)
    return img


def letter_img(path):
    img = ls.letter_seg(path)
    img = cv2.resize(img, dsize=(32, 32), interpolation=cv2.INTER_LINEAR)
    img = np.expand_dims(img, axis=2) 
    img = np.expand_dims(img, axis=0)
    return img


def load_model(path):
    return keras.models.load_model(path)


def predict(model, img):
    return np.argmax(model.predict(img))


def predict_shape(model, img):
    return SHAPE_INDEX[predict(model, img)]


def predict_letter(model, img):
    return chr(predict(model, img) + 65)


In [1]:
from keras.models import Sequential, Model
from keras.layers import Dense, Permute, Conv2D, MaxPooling2D, Flatten, Activation, Dropout
from keras.utils import to_categorical
from PIL import Image
from keras import backend as K
from keras.optimizers import SGD
import numpy as np
import tables

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tables'

In [8]:
def letter_model():
    K.set_image_data_format( 'channels_last' )

    model = Sequential()

    model.add(Permute((1,2, 3), input_shape=(32, 32, 1)))

    model.add(Conv2D(32, kernel_size=(5,5), activation='relu', kernel_initializer='TruncatedNormal', name="conv1"))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    model.add(Dropout(.25))

    model.add(Conv2D(64, kernel_size=(5,5), activation='relu', kernel_initializer='TruncatedNormal', name="conv2"))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    model.add(Dropout(.25))

    model.add(Flatten())
    model.add(Dense(1600, activation='relu', name='fc3'))
    model.add(Dense(26, activation='softmax', name='fc4'))
    
    return model

In [9]:
def shape_model():
    K.set_image_data_format( 'channels_last' )
        
    model = Sequential()

    model.add(Permute((1,2,3), input_shape=(224, 224, 3)))

    model.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu', name="conv1_1"))
    model.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu', name="conv1_2"))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Conv2D(128, kernel_size=(3,3), padding='same', activation='relu', name="conv2_1"))
    model.add(Conv2D(128, kernel_size=(3,3), padding='same', activation='relu', name="conv2_2"))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Conv2D(256, kernel_size=(3,3), padding='same', activation='relu', name="conv3_1"))
    model.add(Conv2D(256, kernel_size=(3,3), padding='same', activation='relu', name="conv3_2"))
    model.add(Conv2D(256, kernel_size=(3,3), padding='same', activation='relu', name="conv3_3"))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Conv2D(512, kernel_size=(3,3), padding='same', activation='relu', name="conv4_1"))
    model.add(Conv2D(512, kernel_size=(3,3), padding='same', activation='relu', name="conv4_2"))
    model.add(Conv2D(512, kernel_size=(3,3), padding='same', activation='relu', name="conv4_3"))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Conv2D(512, kernel_size=(3,3), padding='same', activation='relu', name="conv5_1"))
    model.add(Conv2D(512, kernel_size=(3,3), padding='same', activation='relu', name="conv5_2"))
    model.add(Conv2D(512, kernel_size=(3,3), padding='same', activation='relu', name="conv5_3"))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(Conv2D(4096, kernel_size=(7,7), activation='relu', name='fc6'))
    model.add(Conv2D(4096, kernel_size=(1,1), activation='relu', name='fc7'))
    model.add(Conv2D(11, kernel_size=(1,1),  name='fc8'))
    model.add(Flatten())
    model.add( Activation('softmax') )
    
    return model

In [10]:
def process_data(data_path, type=""):
    if type == "":
        raise ValueError('Please input the type of data (shape or letter)')
    
    label_size = {"shape": 11, "letter": 26}
    
    f = tables.open_file(data_path, mode='r')

    x_train = f.root.train_input[()]
    y_train = f.root.train_labels[()]

    x_test = f.root.test_input[()]
    y_test = f.root.test_labels[()]

    f.close()

    y_train = to_categorical(y_train, label_size[type])
    y_test = to_categorical(y_test, label_size[type])

    return x_train, y_train, x_test, y_test

In [11]:
def train(model, x_train, y_train, x_test, y_test, nEpochs=10, nBatch=32, model_name="model"):
    
    sgd = SGD(lr=0.0001)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x=x_train, y=y_train, batch_size=nBatch, epochs=nEpochs, verbose=1)
    model.evaluate(x=x_test, y=y_test, verbose=1)
    model.save(model_name + '.hdf5')
    return model

In [12]:
def process_letter(addr):
    img = Image.open(addr).convert('L')
    img = img.reshape((32,32))
    img = np.array(img).astype(np.float32)
    img = np.expand_dims(img, axis=0)
    img = np.expand_dims(img, axis=3)
    
    return img

In [13]:
def process_shape(addr):
    img = Image.open(addr).convert('L')
    img = img.reshape((224,224))
    img = np.array(img).astype(np.float32)
    img = np.expand_dims(img, axis=0)

In [14]:
def predict(model, img):
    
    return model.predict(img)

In [15]:
import os
print("hello!")
print(os.getcwd())

hello!
/Users/Andy/Andy/Undergrad/Year_1/uas/drone_code/src/vision/classifier
